## 层和块
首先回顾多层感知机

In [2]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
X = torch.rand(2, 20)
net(X)

tensor([[ 0.0499,  0.0349, -0.1541,  0.2290,  0.2741,  0.1072,  0.3851, -0.3438,
          0.2483, -0.0377],
        [ 0.0677, -0.1356, -0.0606,  0.2773,  0.2038,  0.1366,  0.5117, -0.1355,
          0.1620, -0.0779]], grad_fn=<AddmmBackward0>)

```nn.Sequential```定义了一种特殊的```Module```

## 自定义块

In [7]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 定义隐藏层
        self.hidden = nn.Linear(20, 256)
        # 输出层
        self.out = nn.Linear(256, 10)
        
    def forward(self, X):
        # 前向函数
        print('invoke forward')
        return self.out(F.relu(self.hidden(X)))

## 实例化多层感知机的层， 然后在每次调用正向传播函数时调用这些层

In [8]:
net = MLP()
net(X)

invoke forward


tensor([[ 0.0873, -0.1229,  0.1344, -0.2407,  0.1666, -0.2201,  0.0807,  0.0518,
          0.0960, -0.0243],
        [ 0.1857,  0.0677, -0.0230, -0.2522,  0.1925, -0.1181,  0.0872, -0.0013,
          0.1354,  0.0913]], grad_fn=<AddmmBackward0>)

## 循序块

In [10]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block
            
    def forward(self, X):
        # 此处有序
        for block in self._modules.values():
            X = block(X)
        return X
net= MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0669, -0.1744,  0.0413, -0.1529, -0.0542,  0.1793, -0.1803, -0.0529,
          0.0802, -0.0937],
        [ 0.1971, -0.1017,  0.0746, -0.0062, -0.0143,  0.1495, -0.2605, -0.1055,
          0.0186,  0.0519]], grad_fn=<AddmmBackward0>)

## 在正向传播函数中执行代码

In [14]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(0.0170, grad_fn=<SumBackward0>)

## 混合搭配各种组合块的方法

In [19]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
    
    def forward(self, X):
        return self.linear(self.net(X))

chimera =nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.2879, grad_fn=<SumBackward0>)